In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer

# tr_offenseval_tweets_df = pd.read_csv('offenseval2020-turkish/offenseval-tr-testset-v1/offenseval-tr-testset-v1.tsv', sep="\t")
# tr_offenseval_labels_df = pd.read_csv('offenseval2020-turkish/offenseval-tr-testset-v1/offenseval-tr-labela-v1.tsv', sep=",")
# HSO = pd.read_csv('hatespeech_offensive.csv', sep=',')

OLID = pd.read_csv('OLIDv1.0/olid-training-v1.0.tsv', sep="\t")

# Trainer (has labels)
# trainer_tweets_df = pd.read_csv('extended_test_SOLID/test_a_tweets_all.tsv', sep="\t")
# trainer_labels_df = pd.read_csv('extended_test_SOLID/test_a_labels_all.csv', sep=",")

# Unsupervised learner (no labels)
# learner_tweets_df = pd.read_csv('SOLID_tweets_task_a_distant.tsv', sep="\t")

/home/cymn/miniconda3/envs/nvidia/lib/python3.11/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [4]:
OLID.head()['tweet'].values

array(['@USER She should ask a few native Americans what their take on this is.',
       '@USER @USER Go home you’re drunk!!! @USER #MAGA #Trump2020 👊🇺🇸👊 URL',
       'Amazon is investigating Chinese employees who are selling internal data to third-party sellers looking for an edge in the competitive marketplace. URL #Amazon #MAGA #KAG #CHINA #TCOT',
       '@USER Someone should\'veTaken" this piece of shit to a volcano. 😂"',
       '@USER @USER Obama wanted liberals &amp; illegals to move into red states'],
      dtype=object)

In [5]:

def remap_labels(df, col_name):
    df[col_name] = df[col_name].map({'OFF': 1, 'NOT': 0})
    return df
def remap_HSO_labels(df):
    df['class'] = df['class'].map({2: 0, 1: 1, 0: 1}) 
    return df

def remap_tweets(df, col_name):
    df.loc[:, col_name] = df.loc[:, col_name].str.replace('@USER', '') #Remove mentions (@USER)
    df.loc[:, col_name] = df.loc[:, col_name].str.replace('URL', '') #Remove URLs
    pat = r'(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])' # remove URLS
    df.loc[:, col_name] = df.loc[:, col_name].str.replace(pat, ' ', regex=True)
    df.loc[:, col_name] = df.loc[:, col_name].str.replace('&amp', 'and') #Replace ampersand (&) with and
    df.loc[:, col_name] = df.loc[:, col_name].str.replace('&lt','') #Remove &lt
    df.loc[:, col_name] = df.loc[:, col_name].str.replace('&gt','') #Remove &gt
    df.loc[:, col_name] = df.loc[:, col_name].str.replace('\n',' ') #Remove \n
    df.loc[:, col_name] = df.loc[:, col_name].str.replace('\d+','', regex=True) #Remove numbers
    pat = r'[^\w\s]' # remove the emojis and other special characters
    df.loc[:, col_name] = df.loc[:, col_name].str.replace(pat, '', regex=True)
    #remove white space before and after the tweet
    df.loc[:, col_name] = df.loc[:, col_name].str.strip()
    df.loc[:, col_name] = df.loc[:, col_name].str.lower() #Lowercase
    
    # not needed outside of english
    contractions_dict = {
        r"\bive\b": "i have",
        r"\byoure\b": "you are",
        r"\bitd\b": "it would",
        r"\bcant\b": "cannot",
        r"\bdont\b": "do not",
        r"\bwont\b": "will not",
        r"\btho\b": "though",
        r"\bim\b": "i am",
        r"\bhes\b": "he is",
        r"\bshes\b": "she is",
        r"\bits\b": "it is",
        r"\bwere\b": "we are",
        r"\btheyre\b": "they are",
        r"\bshouldnt\b": "should not",
        r"\bwouldnt\b": "would not",
        r"\bcouldnt\b": "could not",
        r"\bisnt\b": "is not",
        r"\barent\b": "are not",
        r"\bhavent\b": "have not",
        r"\bhasnt\b": "has not",
        r"\bhadnt\b": "had not",
        r"\bwasnt\b": "was not",
        r"\bwerent\b": "were not"
    }
    
    # df[col_name] = df[col_name].replace(contractions_dict, regex=True) # Replace some popular contractions
    
    
    return df

In [6]:
# trainer_labels_df = remap_labels(trainer_labels_df, 'label')
# trainer_tweets_df = remap_tweets(trainer_tweets_df, 'tweet')
# learner_tweets_df = remap_tweets(learner_tweets_df, 'text')
OLID = remap_labels(OLID, 'subtask_a')
OLID = remap_tweets(OLID, 'tweet')
# tr_offenseval_labels_df = remap_labels(tr_offenseval_labels_df, 'label')
# tr_offenseval_tweets_df = remap_tweets(tr_offenseval_tweets_df, 'tweet')

# HSO_df = remap_HSO_labels(HSO)
# HSO_df = remap_tweets(HSO, 'tweet')


In [7]:
OLID.rename(columns={'subtask_a': 'label'}, inplace=True)
# tr_offenseval_df.rename(columns={'subtask_a': 'label'}, inplace=True)
# HSO_df.rename(columns={'class': 'label'}, inplace=True)


In [44]:
# OLID.to_csv('datasets/cleaned_OLID.tsv', sep='\t', index=False)
# trainer_df = trainer_tweets_df.merge(trainer_labels_df, on='id')
# trainer_df = trainer_tweets_df.merge(trainer_labels_df, on='id')
# tr_offenseval_test_df = tr_offenseval_tweets_df.merge(tr_offenseval_labels_df, on='id')
# HSO_df = HSO_df.drop(columns=['count','hate_speech','offensive_language','neither'])

# trainer_df.to_csv('datasets/cleaned_SOLIDtest6K_trainer.tsv', sep='\t', index=False)
# learner_tweets_df.to_csv('datasets/cleaned_SOLID9M_learner.tsv', sep='\t',index=False)
# HSO_df.to_csv('datasets/cleaned_hatespeech_offensive.tsv', sep='\t',index=False)



In [8]:
OLID.head(12)['tweet'].values

array(['she should ask a few native americans what their take on this is',
       'go home youre drunk  maga trump',
       'amazon is investigating chinese employees who are selling internal data to thirdparty sellers looking for an edge in the competitive marketplace  amazon maga kag china tcot',
       'someone shouldvetaken this piece of shit to a volcano',
       'obama wanted liberals and illegals to move into red states',
       'liberals are all kookoo', 'oh noes tough shit',
       'was literally just talking about this lol all mass shootings like that have been set ups its propaganda used to divide us on major issues like gun control and terrorism',
       'buy more icecream',
       'canada doesnt need another cuck we already have enough looneyleft liberals fking up our great country qproofs trudeaumustgo',
       'its not my fault you support gun control',
       'whats the difference between kavanaugh and    one of these men admitted to groping a  year old girl years ago  